In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from sklearn.svm import OneClassSVM
import numpy as np
from scipy import sparse

In [2]:
df_train = pd.read_csv(r'C:\Users\camil\OneDrive\Área de Trabalho\mestrado\treino\r2_bo_train_statements.csv', sep=';')
df_train = df_train.drop(df_train[df_train.Polarity =='other'].index)
df_train.Polarity.replace(('for', 'against'), (1,0), inplace=True)
df_train = df_train.sample(frac=1).reset_index(drop=True)
df_train

,Text,Target,Polarity,Factual,User_ID,Tweet_ID,Tweet_Seq,POS,Mentions,Nouns
0,Querem porque querem diminuir o Bolsonaro! Qua...,r2_bo,1,no,r2_bo_724,_1097684144128061440,737,"[('querem', 'V'), ('porque', 'KS'), ('querem',...",[],{'bolsonaro': 1}
1,"Cortejado por nacionalistas, Bolsonaro gera in...",r2_bo,0,yes,r2_bo_75,_1220385834848980992,1874,"[('cortejado', 'PCP'), ('por', 'PREP'), ('naci...","['uolnoticias', 'uol']","{'uolnoticias': 1, 'bolsonaro': 1, 'nacionalis..."
2,"eu assistindo Simpsons, o Homem presidente é i...",r2_bo,0,no,r2_bo_428,_1261426640837632001,441,"[('eu', 'PROPESS'), ('assistindo', 'V'), ('sim...",[],"{'homem': 1, 'bolsonaro': 1, 'simpsons': 1, 'p..."
3,Eu continuo com a impressão fortíssima que o B...,r2_bo,0,no,r2_bo_533,_1253406007243358208,2246,"[('eu', 'PROPESS'), ('continuo', 'V'), ('com',...",[],"{'atentado': 1, 'impressão': 1, 'mandato': 1, ..."
4,Vocês tem noção que ele fez saudação nazista p...,r2_bo,0,no,r2_bo_72,_1268582048027074565,3845,"[('vocês', 'PROPESS'), ('tem', 'V'), ('noção',...",[],"{'saudação': 1, 'noção': 1, 'bolsonaro': 1}"
...,...,...,...,...,...,...,...,...,...,...
7055,Traição de Bolsonaro dá início à guerra que po...,r2_bo,0,no,r2_bo_75,_1210432296320409600,1071,"[('traição', 'N'), ('de', 'PREP'), ('bolsonaro...",['youtube'],"{'governo': 1, 'youtube': 1, 'guerra': 1, 'tra..."
7056,Minha mãe falando pra eu não desejar a morte d...,r2_bo,0,no,r2_bo_693,_1280302494879547393,4582,"[('minha', 'PROADJ'), ('mãe', 'N'), ('falando'...",[],"{'mãe': 1, 'morte': 1, 'bolsonaro': 1, 'karma'..."
7057,"Aí tu já passa dos limites, falar que é melhor...",r2_bo,0,no,r2_bo_385,_1062484565023952896,3150,"[('aí', 'ADV'), ('tu', 'PROPESS'), ('já', 'ADV...",[],"{'futebol': 1, 'haddad': 1, 'limites': 1}"
7058,Se é o pessoal q votou no Bolsonaro que está c...,r2_bo,0,no,r2_bo_21,_1094668954776281092,747,"[('se', 'KS'), ('é', 'V'), ('o', 'ART'), ('pes...",[],"{'bolsonaro': 1, 'q': 1, 'troféu': 1, 'preconc..."


In [3]:
df_test = pd.read_csv(r'C:\Users\camil\OneDrive\Área de Trabalho\mestrado\Teste\r2_bo_test_statements.csv', sep=';')
df_test = df_test.drop(df_test[df_test.Polarity =='other'].index)
df_test.Polarity.replace(('for', 'against'), (1,0), inplace=True)
df_test = df_test.sample(frac=1).reset_index(drop=True)
df_test

,Text,Target,Polarity,Factual,User_ID,Tweet_ID,Tweet_Seq,POS,Mentions,Nouns
0,ESSE BOLSONARO É UM FILHO DA PUT* E QUEM AINDA...,r2_bo,0,no,r2_bo_308,_1250936760080760832,4917,"[('esse', 'PROADJ'), ('bolsonaro', 'N'), ('é',...",[],"{'filho': 1, 'favor': 1, 'bolsonaro': 1}"
1,esse povo do governo do bolsonaro parece eu nã...,r2_bo,0,no,r2_bo_393,_1124319931842158592,488,"[('esse', 'PROADJ'), ('povo', 'N'), ('do', 'PR...",[],"{'povo': 1, 'governo': 1, 'bolsonaro': 1, 'twi..."
2,"A u não respeito a opinião de fascista, tu me ...",r2_bo,0,no,r2_bo_496,_1047902447296040960,2620,"[('a', 'PREP'), ('u', 'N'), ('não', 'ADV'), ('...",[],"{'opinião': 1, 'caras': 1, 'u': 1, 'discurso':..."
3,"A visita não é o motivo da vergonha, mas sim o...",r2_bo,0,no,r2_bo_671,_1111989272179957760,556,"[('a', 'ART'), ('visita', 'N'), ('não', 'ADV')...",[],"{'vergonha': 1, 'bolsonaro': 1, 'motivo': 1, '..."
4,Esse @ nunca foi apoiador de Jair Messias Bols...,r2_bo,1,no,r2_bo_734,_1264189677168472064,2005,"[('esse', 'PROADJ'), ('@', 'NUM'), ('datenaofi...",['datenaoficial'],"{'datenaoficial': 1, 'picareta': 1, 'máscara':..."
...,...,...,...,...,...,...,...,...,...,...
2349,Vamos dar uma ajudinha!!!Vamos em frente...Aju...,r2_bo,1,no,r2_bo_702,_1130592490887745537,1322,"[(',', 'PU'), ('vamos', 'V'), ('dar', 'V'), ('...",['jairbolsonaro'],"{'presidente': 1, 'jairbolsonaro': 1, 'ajudinh..."
2350,Essa estória já deu! O Presidente luta todos o...,r2_bo,1,no,r2_bo_125,_1309525294097850368,2294,"[('essa', 'PROADJ'), ('estória', 'N'), ('já', ...",[],"{'presidente': 2, 'filósofos': 1, 'balde': 1, ..."
2351,que vontade de morrer com essa tag de mulheres...,r2_bo,0,no,r2_bo_183,_1022609697671716865,2599,"[('que', 'KS'), ('vontade', 'N'), ('de', 'PREP...",[],"{'tag': 1, 'bolsonaro': 1, 'brasil': 1, 'tt': ..."
2352,se eu ganhasse 1 real pra cada vez q um seguid...,r2_bo,0,no,r2_bo_291,_1263619149823184898,962,"[('@', 'NUM'), ('fernanda_seabra', 'N'), ('@',...","['fernanda_seabra', 'abrahamweint']","{'esquerda': 2, 'seguidorzinho': 1, 'fernanda_..."


In [4]:
#carregando unlabeled data
df_u = pd.read_csv(r'C:/Users/camil/OneDrive/Área de Trabalho/mestrado/Unlabeled/bolsonaro.csv')
#df_u = df.sample(frac=1).reset_index(drop=True)
df_u

,Unnamed: 0,Text
0,0,isso me lembra eu anto e bruna discutindo c o ...
1,1,bolsonaro fdp acabando c a maconha esse país t...
2,2,Bolsonaro 2018? — No dia das eleições eu ainda...
3,3,Eu amo o Bolsonaro
4,4,acabou os seiscentao do bolsonaro entao amanha...
...,...,...
31990,31990,Eu conto 0 items dessa lista no governo Bolson...
31991,31991,Não existe comparação nenhuma entre os últimos...
31992,31992,Sendo sincero: seria injusto até comparar a ca...
31993,31993,Eu vou ser sinceros com vcs: eu não acho que n...


In [5]:
rows_train = len(df_train.index)
rows_test = len(df_test.index)
rows_label=rows_train+rows_test

In [6]:
rows_unlabel = len(df_u.index)
rows_unlabel

31995

In [7]:
frames = [df_train.Text, df_test.Text, df_u.Text]
X = pd.concat(frames)
X

0        Querem porque querem diminuir o Bolsonaro! Qua...
1        Cortejado por nacionalistas, Bolsonaro gera in...
2        eu assistindo Simpsons, o Homem presidente é i...
3        Eu continuo com a impressão fortíssima que o B...
4        Vocês tem noção que ele fez saudação nazista p...
                               ...                        
31990    Eu conto 0 items dessa lista no governo Bolson...
31991    Não existe comparação nenhuma entre os últimos...
31992    Sendo sincero: seria injusto até comparar a ca...
31993    Eu vou ser sinceros com vcs: eu não acho que n...
31994    Esse vídeo da reunião dos ministros eh um absu...
Name: Text, Length: 41409, dtype: object

In [8]:
#vetoriza conjunto completo
from sklearn.feature_extraction.text import TfidfTransformer
count_vect = CountVectorizer( )
X_counts = count_vect.fit_transform(X)
X_counts.shape

(41409, 39298)

In [9]:
tf_transformer = TfidfTransformer(use_idf=False).fit(X_counts)
X_tf = tf_transformer.transform(X_counts)
X_tf.shape
X_ori = X_tf
#y_ori = df_train.Polarity
X_ori.shape

(41409, 39298)

In [10]:
x_train = X_ori[0:rows_train,:]
x_test = X_ori[rows_train:rows_train+rows_test,:]
x_test.shape


(2354, 39298)

In [11]:
#y_ori = df.Polarity
y_train = df_train.Polarity
y_test = df_test.Polarity
y_test.shape

(2354,)

In [12]:
#classificação
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
model = LogisticRegression(random_state=0)
model.fit(x_train,y_train)
pred_values = model.predict(x_test)     
acc = accuracy_score(pred_values , y_test)
acc

0.7693288020390824

In [13]:
f1_score(y_test, pred_values, average='macro')

0.7497134834595807

In [14]:
precision_score(y_test, pred_values, average='macro')

0.7738435590850982

In [15]:
recall_score(y_test, pred_values, average='macro')

0.7429826212153798

In [16]:
import eli5
classes = [0,1]
weights = eli5.show_weights(model, feature_names = count_vect.get_feature_names(),  target_names=classes, top = 50)   # ['diag','ctrl'] ???
url = "teste.htm"
with open(url,'wb') as f:
    f.write(weights.data.encode("UTF-8"))

C:\Users\camil\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [17]:
weights

Weight?,Feature
+5.307,presidente
+3.760,nosso
+3.360,mídia
+2.875,está
+2.741,imprensa
+2.724,esquerda
+2.710,parabéns
+2.642,povo
+2.633,stf
+2.595,apoio


In [17]:
#predição dos não-rotulados
X_label = X_ori[rows_label:rows_label+ rows_unlabel]
pred_values_label= model.predict(X_label) 


In [18]:
#avalia a probabilidade de estar certo
prob_label = model.predict_proba(X_label) 
#np.savetxt('C:/Users/camil/OneDrive/Área de Trabalho/mestrado/Unlabeled/prob.csv', prob_label, delimiter=';')

In [19]:
#seleciona os itens que vão entrar no conjunto de treino
threshold = 0.01
t_size = int(threshold*rows_unlabel) #quanto vai adicionar a cada iteração
rows_train = x_train.shape[0]
rows_train

7060

In [20]:
reliable_pred_favor, reliable_pred_against  = [], []
for x_id, (prob, pred) in enumerate(zip(prob_label, pred_values_label)):
    #se 1, inclui no favor; senão, no against
    #incluir id, probabilidade e valor predito (1 ou 0)
    (reliable_pred_favor if (pred_values_label[x_id]) else reliable_pred_against).append((x_id, prob_label[x_id,pred_values_label[x_id]], pred_values_label[x_id]))
    

In [21]:
#coloca em ordem decrescente de confiabilidade a partir da probabilidade (coluna 2)
reliable_pred_favor = sorted(reliable_pred_favor, key=lambda x: x[1], reverse=True)
reliable_pred_against = sorted(reliable_pred_against, key=lambda x: x[1], reverse=True)

In [22]:
y= np.asarray(y_train)
index_list=[x for x in range(0, rows_train)] #indices para o novo X a partir do X original
iteractions = 0
position = 0
index_labels=[x for x in range(rows_label, rows_label+ rows_unlabel)]

while iteractions < 0.99:
    iteractions = iteractions + threshold
    for i in range(int(t_size/2)):
        #pega um a favor
        index = reliable_pred_favor[position][0] #descobre a posição no X_label 
        index_list.append(index+rows_label) #inclui na lista de indices
        y = np.append(y, 1) #empilha o y
        index_labels.remove(index+rows_label)
        #pega um contra
        index = reliable_pred_against[position][0] #descobre a posição no X_label 
        index_list.append(index+rows_label) #inclui na lista de indices
        y = np.append(y, 0) #empilha o y
        index_labels.remove(index+rows_label)
        #anda uma posição nos dois 
        position+=1

    X_train_novo = X_ori[index_list,:]#atualiza o X rotulado com os índices que estão em index_list
    X_label = X_ori[index_labels,:] #pega só os índices que sobraram
    rows_unlabel = X_label.get_shape()[0] #novo número de linhas sem rótulo
    rows_label = X_train_novo.get_shape()[0] + rows_test #linhas com rótulo novas + as de teste que estão guardadas
    X_ori = sparse.vstack([X_train_novo, x_test, X_label]) #empilha treino, teste e unlabelled
    index_labels=[x for x in range(rows_label, rows_label+ rows_unlabel)] #atualiza os unlabelled
    position = 0

   # x_train_novo, x_test_novo, y_train_novo, y_test_novo = train_test_split(
        #X_novo,
        #np.asarray(y),
        #test_size = 0.25,
        #random_state = 123,
       # shuffle = True,
        #stratify = np.asarray(y),
    #)

    model_novo = LogisticRegression(random_state=0)
    model_novo.fit(X_train_novo,y)
    #testa nos conjuntos originais
    pred_values_novo = model_novo.predict(x_test)     
    acc = accuracy_score(pred_values_novo , y_test)
    print("iteraction:",iteractions, "accuracy:", acc)
    
    pred_values_label= model_novo.predict(X_label) 
    prob_label = model.predict_proba(X_label) 
    reliable_pred_favor, reliable_pred_against  = [], []
    for x_id, (prob, pred) in enumerate(zip(prob_label, pred_values_label)):
        (reliable_pred_favor if (pred_values_label[x_id]) else reliable_pred_against).append((x_id, prob_label[x_id,pred_values_label[x_id]], pred_values_label[x_id]))
    
    reliable_pred_favor = sorted(reliable_pred_favor, key=lambda x: x[1], reverse=True)
    reliable_pred_against = sorted(reliable_pred_against, key=lambda x: x[1], reverse=True)    
    
    index_labels=[x for x in range(rows_label, rows_label+ rows_unlabel)]

iteraction: 0.01 accuracy: 0.7684791843670349
iteraction: 0.02 accuracy: 0.7659303313508921
iteraction: 0.03 accuracy: 0.7655055225148684
iteraction: 0.04 accuracy: 0.7646559048428208
iteraction: 0.05 accuracy: 0.7727272727272727
iteraction: 0.060000000000000005 accuracy: 0.7646559048428208
iteraction: 0.07 accuracy: 0.7655055225148684
iteraction: 0.08 accuracy: 0.7582837723024639
iteraction: 0.09 accuracy: 0.7650807136788446
iteraction: 0.09999999999999999 accuracy: 0.7587085811384877
iteraction: 0.10999999999999999 accuracy: 0.7689039932030586
iteraction: 0.11999999999999998 accuracy: 0.7612574341546304


C:\Users\camil\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


iteraction: 0.12999999999999998 accuracy: 0.7625318606627017
iteraction: 0.13999999999999999 accuracy: 0.7684791843670349


C:\Users\camil\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


iteraction: 0.15 accuracy: 0.7582837723024639
iteraction: 0.16 accuracy: 0.7663551401869159
iteraction: 0.17 accuracy: 0.7659303313508921
iteraction: 0.18000000000000002 accuracy: 0.7570093457943925


...

C:\Users\camil\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


iteraction: 0.9700000000000006 accuracy: 0.699660152931181


C:\Users\camil\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


iteraction: 0.9800000000000006 accuracy: 0.6830926083262532
iteraction: 0.9900000000000007 accuracy: 0.6677994902293968


C:\Users\camil\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [23]:
len(reliable_pred_against)

321